# __<u>Классификация токенов</u>__

## __0. Импорт__

In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [47]:
from transformers import pipeline
# from huggingface_hub import list_models
from transformers import AutoTokenizer, AutoModelForTokenClassification
from huggingface_hub import model_info
import torch

## __1. Задачи NER (Named Entity Recognition)__

### __1.1. model_1 - <font color='lightgreen'>dbmdz/bert-large-cased-finetuned-conll03-english</font>__  
https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english

#### Входные данные

In [67]:
# Проверяем, существует ли подходящая
# предобученная модель (если есть информация - модель существует)
model_name_1 = "dbmdz/bert-large-cased-finetuned-conll03-english"
info_1 = model_info(model_name_1)
print(info_1)

ModelInfo(id='dbmdz/bert-large-cased-finetuned-conll03-english', author='dbmdz', sha='4c534963167c08d4b8ff1f88733cf2930f86add0', created_at=datetime.datetime(2022, 3, 2, 23, 29, 5, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2023, 9, 6, 22, 17, 56, tzinfo=datetime.timezone.utc), private=False, disabled=False, downloads=1444675, downloads_all_time=None, gated=False, gguf=None, inference='warm', likes=78, library_name='transformers', tags=['transformers', 'pytorch', 'tf', 'jax', 'rust', 'safetensors', 'bert', 'token-classification', 'autotrain_compatible', 'endpoints_compatible', 'region:us'], pipeline_tag='token-classification', mask_token='[MASK]', card_data=None, widget_data=[{'text': 'My name is Wolfgang and I live in Berlin'}, {'text': 'My name is Sarah and I live in London'}, {'text': 'My name is Clara and I live in Berkeley, California.'}], model_index=None, config={'architectures': ['BertForTokenClassification'], 'model_type': 'bert', 'tokenizer_config': {}}, t

In [99]:
# Чтобы посмотреть, на определение каких типов сущностей настроена модель
# надо загрузить ее через AutoModelForTokenClassification и обратиться
# к результату ее вызова
model_11 = AutoModelForTokenClassification.from_pretrained(model_name_1)
labels_11 = model_11.config.id2label
print("Метки сущностей:", labels_11)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Метки сущностей: {0: 'O', 1: 'B-MISC', 2: 'I-MISC', 3: 'B-PER', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-LOC', 8: 'I-LOC'}


In [100]:
len(labels_11)

9

In [ ]:
text_1 = "My name is Sylvain and I work at Hugging Face in Brooklyn."

##### Итого: Модель определяет 4 типа сущностей для задачи NER:  

<small>- `MISC` – Разные (Miscellaneous), любые другие именованные сущности, не попадающие в предыдущие категории
- `PER` – Имена людей (Person)  
- `LOC` – Географические объекты (Location), такие как города, страны, континенты  
- `ORG` – Организации (Organization), включая компании, спортивные команды, правительства</small>

#### A - pipline

In [ ]:
# Создаем преднастроенный пайплайн (цепочку обработки данных) для задачи NER
token_classifier_1 = pipeline(
    "token-classification", model=model_name_1, aggregation_strategy="simple"
)

##### Comments about: <font color='orange'>aggregation_strategy="simple"</font> -

<small>oпределяет, как объединять предсказания модели. В BERT одно слово может быть разбито на несколько подслов (subwords), например: "Hugging Face" → ["Hug", "##ging", "Face"]. Чтобы не получать предсказания для каждого подслова отдельно, `aggregation_strategy="simple"` объединяет их в одно слово.  
Без агрегации модель вернет что-то вроде:
```
[ {'word': 'Hug', 'entity': 'ORG', 'score': 0.98},
  {'word': '##ging', 'entity': 'ORG', 'score': 0.97},
  {'word': 'Face', 'entity': 'ORG', 'score': 0.99}]
```  
Другие стратегии агрегации:
- `"none"` — не объединяет подслова (отдельные предсказания для каждого токена).  
- `"first"` — берет метку только для первого подслова.  
- `"average"` — усредняет предсказания.  
- `"max"` — берет метку с наибольшей вероятностью.</small>  

##### Тест

In [71]:
result = token_classifier_1(text_1)
print(result)
words = [entry['word'] for entry in result] # Фильтруем только слова
print(words)
words_and_probabilities = [(entry['word'], entry['score']) for entry in result] # Фильтруем слова и вероятности
print(words_and_probabilities)

[{'entity_group': 'PER', 'score': 0.9981694, 'word': 'Sylvain', 'start': 11, 'end': 18}, {'entity_group': 'ORG', 'score': 0.9796019, 'word': 'Hugging Face', 'start': 33, 'end': 45}, {'entity_group': 'LOC', 'score': 0.9932106, 'word': 'Brooklyn', 'start': 49, 'end': 57}]
['Sylvain', 'Hugging Face', 'Brooklyn']
[('Sylvain', 0.9981694), ('Hugging Face', 0.9796019), ('Brooklyn', 0.9932106)]


#### B - AutoModelForTokenClassification

In [73]:
# model_11 = AutoModelForTokenClassification.from_pretrained(model_name_1)
# (мы это сделали выше, когда получали информацию об модели)
tokenizer_11 = AutoTokenizer.from_pretrained(model_name_1)

<font color='yellow'>Дальше возникает проблема.  В отличии от работы через pipline, здесь приходится самостоятельно обрабатывать процедуры токенизации. И в результате их
- токенизация:  `inputs_11 = tokenizer_11(text_1, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)`   
- связь токенов с исходными словами: `word_ids = inputs_11.word_ids()`  
- предсказание
```
with torch.no_grad()`:  # отключаем градиенты  
    outputs = model_11(**inputs_11)
```
- далее смотрим на логиты (предсказания) для каждого токена
```
logits = outputs.logits
print("logits:", logits)
```   
и обнаруживаем, что нет максимальных значений на начальных частях слова (а слова разбиты на токены), в результате метки "B-" не регистрируются, только метки продолжения слова "I-". Далее надо все это в ручную исправлять, что затратно...</font>  

##### Тест

In [101]:
# # В отличии от работы через pipline, здесь нам надо самостоятельно токенизировать текст
# inputs_11 = tokenizer_11(text_1, return_tensors="pt", truncation=True, padding=True, is_split_into_words=False)
# word_ids = inputs_11.word_ids()  # Связь токенов с исходными словами
# # Получаем предсказания
# with torch.no_grad():  # отключаем градиенты, так как мы только делаем предсказание
#     outputs = model_11(**inputs_11)

# # Логиты (предсказания) для каждого токена
# logits = outputs.logits
# print("logits:", logits)

# # Получаем метки для предсказанных классов
# labels = model_11.config.id2label
# print("labels:",  labels)
# # Есть {0: 'O', 1: 'B-MISC', 2: 'I-MISC', 3: 'B-PER', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-LOC', 8: 'I-LOC'}

# # Получаем индексы максимальных логитов (предсказания классов)
# predicted_ids = torch.argmax(logits, dim=2).squeeze().tolist()
# print(f"Predicted IDs: {predicted_ids}")
# # не совпадают, нет 1, 3, 5, 7 - [0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]

# # # Преобразуем токены и метки в сущности (агрегация как в pipeline)
# tokens = tokenizer_11.convert_ids_to_tokens(inputs_11["input_ids"].squeeze())
# print(tokens)


logits: tensor([[[ 8.7508, -2.2626, -1.5300, -2.2889, -0.6513, -2.0016, -0.0112,
          -2.0860,  0.3335],
         [ 8.4973, -2.3986, -1.3582, -2.7887,  0.7575, -1.8873,  0.4344,
          -1.9900, -0.3397],
         [ 9.4719, -2.2261, -0.9849, -2.6116,  0.1219, -2.0627, -0.1259,
          -1.8758, -0.0609],
         [ 9.8670, -2.2175, -1.3125, -2.4866, -0.2550, -1.8536,  0.0856,
          -1.7520, -0.6437],
         [-0.2011, -2.1873, -1.5316, -2.7110,  8.4025, -2.4168, -0.6980,
          -3.0337, -0.0997],
         [ 0.1065, -2.0520, -1.4787, -2.8139,  7.4525, -2.8399, -0.0626,
          -3.3666, -0.4683],
         [ 0.5985, -2.2538, -1.1926, -3.0111,  7.0070, -2.8675,  0.3492,
          -3.3129, -0.2878],
         [-0.0584, -2.2660, -1.4335, -3.1940,  8.3225, -2.6212, -0.0348,
          -2.9780, -0.2957],
         [ 9.6889, -2.4281, -1.5653, -2.5225, -0.9693, -1.5668,  0.4285,
          -1.9413, -0.6774],
         [ 9.0116, -2.1216, -1.4140, -2.6964,  0.2728, -1.7851,  0.3635,
 

### 1.2. Model - <font color='lightgreen'>tner/roberta-large-wnut2017</font>  
https://huggingface.co/tner/roberta-large-wnut2017

#### Входные данные

In [103]:
# Проверяем, существует ли подходящая
# предобученная модель (если есть информация - модель существует)
model_name_2 = "tner/roberta-large-wnut2017"
info_2 = model_info(model_name_2)
print(info_2)

ModelInfo(id='tner/roberta-large-wnut2017', author='tner', sha='e2e15e43657d08a44459430d8413563e709622af', created_at=datetime.datetime(2022, 8, 9, 23, 12, 35, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2022, 9, 26, 14, 16, 19, tzinfo=datetime.timezone.utc), private=False, disabled=False, downloads=1059, downloads_all_time=None, gated=False, gguf=None, inference=None, likes=0, library_name='transformers', tags=['transformers', 'pytorch', 'roberta', 'token-classification', 'dataset:wnut2017', 'model-index', 'autotrain_compatible', 'endpoints_compatible', 'region:us'], pipeline_tag='token-classification', mask_token='<mask>', card_data={'base_model': None, 'datasets': ['wnut2017'], 'eval_results': [EvalResult(task_type='token-classification', dataset_type='wnut2017', dataset_name='wnut2017', metric_type='f1', metric_value=0.5375139977603584, task_name='Token Classification', dataset_config=None, dataset_split=None, dataset_revision=None, dataset_args='wnut2017', metri

In [104]:
# Чтобы посмотреть, на определение каких типов сущностей настроена модель
# надо загрузить ее через AutoModelForTokenClassification и обратиться
# к результату ее вызова
model_22 = AutoModelForTokenClassification.from_pretrained(model_name_2)
labels_22 = model_22.config.id2label
print("Метки сущностей:", labels_22)

Метки сущностей: {0: 'B-corporation', 1: 'B-creative-work', 2: 'B-group', 3: 'B-location', 4: 'B-person', 5: 'B-product', 6: 'I-corporation', 7: 'I-creative-work', 8: 'I-group', 9: 'I-location', 10: 'I-person', 11: 'I-product', 12: 'O'}


In [105]:
len(labels_22)

13

##### Итого: Модель определяет 6 типов сущностей для задачи NER:  


- <small>`person` – Имена людей  
- `location` – Любые географические объекты (города, страны, улицы, природные объекты)
- `corporation` – Организации и компании  
- `product` – Продукты (софт, устройства, транспортные средства)  
- `creative-work` – Произведения искусства (фильмы, книги, песни и т. д.)  
- `group` – Музыкальные группы, спортивные команды, сообщества  
Эта модель ориентирована на обработку шумных данных (например, текстов из соцсетей и форумов), где встречаются неформатированные и нестандартные именованные сущности.</small>

#### A - pipline

In [108]:
# Создаем преднастроенный пайплайн (цепочку обработки данных) для задачи NER
token_classifier_2 = pipeline(
    "token-classification", model_name_2, aggregation_strategy="simple"
)

Device set to use cpu


##### Тест

In [131]:
text_2 = "My name is Shaps.I work in Greenlife in Antaly."
text_22 = "My name is Shaps.I work in Greenlife in Antaly. I create paintings."

In [133]:
result = token_classifier_1(text_2)
# print(result)
# words = [entry['word'] for entry in result] # Фильтруем только слова
# print(words)
words_and_probabilities = [(entry['word'], entry['entity_group'], entry['score']) for entry in result] # Фильтруем слова и вероятности
print(words_and_probabilities)

[('Shaps', 'PER', 0.9949458), ('Greenlife', 'ORG', 0.9777943), ('Antaly', 'LOC', 0.9793875)]
